<a href="https://colab.research.google.com/github/samgregson/prompt_optimizer/blob/main/examples/basic_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup for Colab

In [8]:
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  !pip install git+https://github.com/samgregson/prompt_optimizer.git
  !pip install openai
  import os
  import openai
  from google.colab import userdata
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
  openai.api_key = os.getenv("OPENAI_API_KEY")

## Basic steps:
1. Add a `@llm_node` decorator to an LLM function
2. Create an evaluation template
3. Define your "training" data
4. Set up the optimizer
5. Check the results!

### Imports:

In [ ]:
from textwrap import dedent
from openai import OpenAI
from prompt_optimizer import Optimizer, llm_node, OpenAIAdapter

### 1. LLM 'Program' to optimise:

In [ ]:
client = OpenAI()  # ensure you have OPENAI_API_KEY in environmental variables / secrets

system_prompt = "you are a helpful assistent"

# Add a `llm_node` decorator around a function which takes a prompt to be optimised
# the input should be a component to be optimised (`system_prompt`` in this case)
@llm_node(system_prompt=system_prompt)
def answer_query(query: str, system_prompt: str = system_prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query},
        ],
    )
    return response.choices[0].message.content.strip()

### 2. Eval Template:

In [ ]:
# Define a template for evaluating the final answer
# you may use {program_ouput} and any other custom key word arguments (kwargs)
exact_match_evaluator = dedent(
    """
    Your task is to judge the quality if the response given by an AI assistent.
    Below are the <question>, <assistent_response> and <expected_answer>.
    A correct answer would be an exact string match between the <assistent_response> and the
    <expected_answer>

    <question>{query}</question>
    <assistent_response>{program_ouput}</assistent_response>
    <expected_answer>{golden_answer}</expected_answer>
    """
)

### 3. Data:

In [ ]:
# Define your dataset, this could have any number of items in the dictionary
# and contain any number of examples
data = [{
    "query": "what is the capital of France?",
    "golden_answer": "<answer>Paris</answer>"
}]

### 4. Optimize:

In [13]:
# Instantiate an optimizer and pass in an `LLMCallable`
# (in this case using the OpenAIAdapter)
llm = OpenAIAdapter(client)
optimizer = Optimizer(llm)

# Optimize
optimizer.optimize(
    iterations=1,
    program_func=answer_query,
    evaluation_template=exact_match_evaluator,
    data=data,
)

{'answer_query': {'system_prompt': <prompt_optimizer.prompt_optimizer.OptimizableComponent at 0x7bec5d86cf10>}}

### 5. Check the results:

In [14]:
# Check the results
result = answer_query("what is the capital of England?").value

print("#### RESULT: ####")
print(result)
# ---- expected answer: ----
# <answer>London</answer>

print()
print("#### PROMPT INFO: ####")
print(optimizer.get_prompt_info())
# ---- expected answer: ----
# Node: `answer_query`
#  system_prompt: Provide the answer strictly in the required format, enclosing your response within <answer></answer> tags. Ensure that the content is clear, concise, and directly addresses the question while maintaining a helpful tone.

#### RESULT: ####
<answer>London</answer>

#### PROMPT INFO: ####
Node: `answer_query`
  system_prompt: Respond succinctly and directly to questions. Provide answers in the format <answer>your_answer</answer> without additional wording. Ensure that your response strictly adheres to the format and content specified in the question without extraneous information. Always answer questions directly and avoid unnecessary elaboration.

